<font color='blue'> Привет!) Поздравляю с первым самостоятельным проектом. Ты проделала большую работу.   Далее в файле ты можешь найти мои комменатрии, выделенные синим. Пожайлуста, постарайся учесть их в дальнейших работах или доработках.</font>

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')

In [2]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Вывод

В столбцах таблицы 'days_employed'(общий трудовой стаж в днях) и 'total_income'(ежемесячный доход) не все строки заполены, а также эти столбцы имеют формат float64, вещественные числа неточны и могут в дальнейшем привести к ошибкам

<font color='blue'> Хороший анализ сырых данных.</font>

### Шаг 2. Предобработка данных

### Обработка пропусков

Проверим столбцы на пропуски

In [3]:
print(data.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


В 2174 случаях нет сведения о доходах и опыте. В столбце 'days_employed' для ответа на поставленные вопросы необходимости нет. Разберемся со столбцом 'total_income', для этого выясним какие источники дохода есть у клиентов. Посмотрим какие есть категории клиентов по типу занятости:

In [4]:
print(data['income_type'].unique())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


Клиенты со статусом 'безработный', 'студент' и 'в декрете' могут не иметь реального месячного дохода, посчитаем их количество 

In [5]:
print(data['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64


Всего 4 клиента могут не иметь дохода, следовательно пропуски скорей всего обусловлены непредоставленными данными о доходах.

Проведем проверку есть ли доход у этих клиентов.

In [6]:
print(data[data['income_type'] == 'студент'])

      children  days_employed  dob_years education  education_id  \
9410         0    -578.751554         22    высшее             0   

              family_status  family_status_id gender income_type  debt  \
9410  Не женат / не замужем                 4      M     студент     0   

      total_income                                 purpose  
9410  98201.625314  строительство собственной недвижимости  


In [7]:
print(data[data['income_type'] == 'в декрете'])

       children  days_employed  dob_years education  education_id  \
20845         2   -3296.759962         39   СРЕДНЕЕ             1   

         family_status  family_status_id gender income_type  debt  \
20845  женат / замужем                 0      F   в декрете     1   

       total_income     purpose  
20845  53829.130729  автомобиль  


In [8]:
print(data[data['income_type'] == 'безработный'])

       children  days_employed  dob_years education  education_id  \
3133          1  337524.466835         31   среднее             1   
14798         0  395302.838654         45    Высшее             0   

          family_status  family_status_id gender  income_type  debt  \
3133    женат / замужем                 0      M  безработный     1   
14798  гражданский брак                 1      F  безработный     0   

        total_income                  purpose  
3133    59956.991984  покупка жилья для сдачи  
14798  202722.511368             ремонт жилью  


У выбранных категорий есть ежемесячный доход, значит пропуски в данных случайны.

Заменим медианами ежемесячного дохода пропущенные значения в соответствии в каждым типом занятости.

In [9]:
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))

In [10]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Вывод

Были обнаружены пропуски в данных о ежемесячном доходе 2174 клиентов, это 10% случаев. Данная информация нужна для ответа на вопрос влияния уровня дохода на возврат кредита в срок. При невозможности получения недостающих данных, лучше заменить пропуски средним значением дохода в соответствии с типом занятости клиента.

<font color='blue'> А в стаже решила не саполнять пропуски? Ты не будешь использовать этот столбец в анализе? Тогда лучше еще раз это подчеркнуть и вообще исключить столбец из рабочего датасэта.</font>

### Замена типа данных

Переведём столбец total_income в формат int

In [11]:
data['total_income'] = data['total_income'].astype('int')

In [12]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB
None


### Вывод

Стоблец total_income содержал данные типа float64, что неудобно для расчетов и приводит к ошибкам. В нашей задаче данными после запятой можно пренебречь. Целесообразно перевести в целочисленное значение.

<font color='blue'> Хорошо, все те же вопросы про стаж.</font>

### Обработка дубликатов

Проверим всю таблицу на наличие полностью идентичных строк

In [13]:
print('Дубликатов', data.duplicated().sum())

Дубликатов 54


Удалим дубликаты без создания нового столбца и присвоим новые индексы строкам

In [14]:
data = data.drop_duplicates().reset_index(drop=True)

In [15]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
children            21471 non-null int64
days_employed       19351 non-null float64
dob_years           21471 non-null int64
education           21471 non-null object
education_id        21471 non-null int64
family_status       21471 non-null object
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        21471 non-null int64
purpose             21471 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB
None


### Вывод

В нашей таблице нет столбца идентификационного номера клиента, следовательно обнаруженные и удаленные дубликаты могут быть не истинными, а появившимися в следствие случайных совпадений характеристик по столбцам. Можно удалить или не удалять эти дубликаты, по сравнению с общим объемом выборки вклад их мал.

<font color='blue'> Посмотри на столбец education. Есть ли там скрытые дубликаты? Их надо удалить в первую очередь, а потом проделать удаление дубликатов, которое ты выволнила (это было правильно сделано).</font>

### Лемматизация

Определим уникальные значения столбца purpose

In [16]:
print(data['purpose'].value_counts())

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Лемматизируем столбец purpose

In [17]:
from pymystem3 import Mystem
m = Mystem()
def lemma(i):
    lemmas = m.lemmatize(i)
    return lemmas

In [18]:
data['purpose_lemma'] = data['purpose'].apply(lemma)

In [20]:
print(data.head(20))

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

### Вывод

Цели получения кредита у клиентов схожи: проведение свадьбы, получение образования, покупка автомобиля, операции с коммерческой недвижимостью и операции с жилой недвижимостью. Для дальнейшей обработки и ответа на поставленную задачу цели необходимо разбить по категориям.

<font color='blue'>Хорошо, лемматизация выполнена правильно - это самая сложная часть проекта. Кстати попробуй выводить таблтчки без print это по-моему посимпатичнее. Смотри.</font>

In [21]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]"
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]"
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]"
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]"


<font color='blue'> print можно опускать не везде. По умолчанию выводится только последняя строка в ячейке. Но зато так можно выводить и info и describe и все остальное.</font>

### Категоризация данных

Для категоризации целей кредита добавим столбец purpose_id, в котором разобьём цели на категории: сыграть свадьбу - свадьба, получить образование - образование, купить автомобиль - автомобиль, провести операции с коммерческой недвижимостью - коммерческая недвижимость, операции с жильём или жилой невижимостью- жильё. Категоризацию проведем по ключевым словам.

In [20]:
def purpose_id(words):
    if 'свадьба' in words:
        return 'свадьба'
    if 'образование' in words:
        return 'образование'
    if 'автомобиль' in words:
        return 'автомобиль'
    if 'коммерческий' in words:
        return 'коммерческая недвижимость'
    return 'жильё'

data['purpose_id'] = data['purpose_lemma'].apply(purpose_id)
print(data['purpose_id'].value_counts())
            

жильё                        9502
автомобиль                   4308
образование                  4014
свадьба                      2335
коммерческая недвижимость    1312
Name: purpose_id, dtype: int64


Для решения поставленных задач не обходимо категоризовать данные в столбце 'total_income'.
Расчитаем 1й и 3й квартили по доходу всех клиетов и медиану:

In [21]:
q_25_total_income = data['total_income'].quantile(0.25)

In [22]:
q_75_total_income = data['total_income'].quantile(0.75)

In [23]:
median = data['total_income'].median()

Теперь доход можно разделить на 4 категории: низкий, ниже среднего, выше среднего, высокий:

In [24]:
def total_income_type(total_income):
    if total_income < q_25_total_income:
        return 'низкий'
    if q_25_total_income < total_income < median:
        return 'ниже среднего'
    if median < total_income < q_75_total_income:
        return 'выше среднего'
    return 'высокий'

In [25]:
data['total_income_type'] = data['total_income'].apply(total_income_type)

In [26]:
print(data['total_income_type'].value_counts())

высокий          6445
низкий           5368
выше среднего    5245
ниже среднего    4413
Name: total_income_type, dtype: int64


### Вывод

- Чаще всего клиенты берут кредит на покупку недвижимости, в равных случаях на покупку автомобиля и проведение свадьбы.
- Клиенты с высоким уровнем дохода чаще берут кредит.

<font color='blue'> Отлично, категоризация выполнена верно. Если отвлечся от вопросов, которые непосредственно представлены в проекте, то можно еще ввести категоризацию по возрасту и проверить просрочки кредитиов по эти категориям.</font>

### Шаг 3. Ответьте на вопросы

#### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для начала проверим столбец 'children':

In [27]:
print(data['children'].value_counts())

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


В данных закрались ошибки: нельзя уйти в минус по количеству детей, и сильно сомнительно, что у 76 человек их ровненько 20 штук, скорей всего ноль лишний и речь идёт о двух детях.

In [28]:
data['children'] = data['children'].abs()
print(data['children'].value_counts())

0     14107
1      4856
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64


Должников по детям больше нет, остались излишне плодовитые

In [29]:
def children_true(children1):
    if children1 == 20:
        return 2
    return children1
data['children'] = data['children'].apply(children_true)
print(print(data['children'].value_counts()))

0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64
None


<font color='blue'> Записи типа -1 и 20 лучше просто исключить из рассмотрения, а не заменять, потому что ты наверняка не знаешь причину ошибки. Может у семьи действительно 20 приемных детей, а не 2? А может их 0 и 2 случайно прописалась? У нас нет достаточно оснований, чтоб записывать их в 2. Также и с -1. В таких случаях лучше исключить небольшой процент записей, чем портить точные данные </font>

Всё, теперь похоже на правду.

Сгруппируем таблицу data  по количеству детей:

In [30]:
children = data.groupby('children')['children'].count()
print(children)

children
0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64


Сгруппируем клиентов по количеству детей и информации о предудыщих задолженностях.

In [31]:
children_debt = data.groupby('children')['debt'].sum()
print(children_debt)

children
0    1063
1     445
2     202
3      27
4       4
5       0
Name: debt, dtype: int64


Узнаем процентное отношение взявших кредит и тех у кого были задолженности в зависимости от количества детей.

In [32]:
children_scoring = (children_debt / children) * 100
print(children_scoring)

children
0    7.535266
1    9.163921
2    9.492481
3    8.181818
4    9.756098
5    0.000000
dtype: float64


### Вывод

 * Клиенты у которых нет детей, отдают кредит в срок чаще, чем те у кого есть дети 
 * Реже всего отдают кредит в срок семьи с четырьмя детьми 
 * Однако те, у кого три ребенка ответственней выплачивают задолженность по сравнению с семьями с иным количеством детей
 * Выборка для семей с 5-ю детьми слишком мала для оценки

<font color='blue'> Хорошо, все правильно рассчитано.</font>

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сгруппируем таблицу data  по семейному положению:

In [33]:
family_status = data.groupby('family_status')['family_status'].count()
print(family_status)

family_status
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
гражданский брак          4163
женат / замужем          12344
Name: family_status, dtype: int64


Сгруппируем клиентов по семейному положению и информации о предудыщих задолженностях.

In [34]:
family_status_debt = data.groupby('family_status')['debt'].sum()
print(family_status_debt)

family_status
Не женат / не замужем    274
в разводе                 85
вдовец / вдова            63
гражданский брак         388
женат / замужем          931
Name: debt, dtype: int64


Узнаем процентное отношение взявших кредит и тех у кого были задолженности в зависимости от их семейного положения.

In [35]:
family_status_scoring = (family_status_debt / family_status) * 100
print(family_status_scoring)

family_status
Не женат / не замужем    9.750890
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.320202
женат / замужем          7.542126
dtype: float64


<font color='blue'> Здесь все хорошо </font>

### Вывод

- Клиенты у которых есть или были официально зарегистрированные отношения отдают кредит в срок чаще, чем те, кто никогда не состоял в официальном браке
- клиенты, потерявшие своего партнёра, отдают кредит в срок чаще всех остальных категорий заемщиков 

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сгруппируем таблицу data  по уровню дохода:

In [36]:
total_income_type = data.groupby('total_income_type')['total_income_type'].count()
print(total_income_type)

total_income_type
высокий          6445
выше среднего    5245
ниже среднего    4413
низкий           5368
Name: total_income_type, dtype: int64


Сгруппируем клиентов по уровню дохода и информации о предудыщих задолженностях.

In [37]:
total_income_type_debt = data.groupby('total_income_type')['debt'].sum()
print(total_income_type_debt)

total_income_type
высокий          481
выше среднего    448
ниже среднего    385
низкий           427
Name: debt, dtype: int64


Узнаем процентное отношение взявших кредит и тех у кого были задолженности в зависимости от их уровня дохода.

In [38]:
total_income_type_scoring = (total_income_type_debt / total_income_type) * 100
print(total_income_type_scoring)

total_income_type
высокий          7.463150
выше среднего    8.541468
ниже среднего    8.724224
низкий           7.954545
dtype: float64


<font color='blue'> Отлично, да, хорошо что сгруппировала доход по квантилям. </font>

### Вывод

- Клиенты с высоким и низким уровнем дохода отдают кредит в срок чаще, чем те, у кого ежемесячный доход близок к средним значениям.

#### Как разные цели кредита влияют на его возврат в срок?

Сгруппируем таблицу data по цели кредита:

In [39]:
purpose = data.groupby('purpose_id')['purpose_id'].count()
print(purpose)

purpose_id
автомобиль                   4308
жильё                        9502
коммерческая недвижимость    1312
образование                  4014
свадьба                      2335
Name: purpose_id, dtype: int64


Сгруппируем клиентов по цели кредита и информации о предудыщих задолженностях.

In [40]:
purpose_debt = data.groupby('purpose_id')['debt'].sum()
print(purpose_debt)

purpose_id
автомобиль                   403
жильё                        683
коммерческая недвижимость     99
образование                  370
свадьба                      186
Name: debt, dtype: int64


Узнаем процентное отношение взявших кредит и тех, у кого были задолженности, в зависимости от цели.

In [41]:
purpose_scoring = (purpose_debt / purpose) * 100
print(purpose_scoring)

purpose_id
автомобиль                   9.354689
жильё                        7.187960
коммерческая недвижимость    7.545732
образование                  9.217738
свадьба                      7.965739
dtype: float64


<font color='blue'> Здесь тоже все правильно </font>

### Вывод

- Кредиты, взятые на покупку автомобиля и на получение образования отдаются реже
- Чаще всего возвращают в срок кредиты на жильё

### Шаг 4. Общий вывод

Гипотезы:
- семейное положение влияет на факт погашения кредита в срок
- количество детей клиента влияет на факт погашения кредита в срок
#### Общие выводы:

Наличие семьи влияет на сроки возвращения кредита: клиенты в официально зарегистрированных отношениях или когда-либо в них состоявшие отдают кредит в срок чаще, при этом бездетные клиенты меньше имеют задолженностей по займам, чем те у кого их один и более.
В результате обе гипотезы подтверждены.

<font color='blue'> Проект в целом сделан очень хорошо. Постарайся поправить несколько небольших замечаний, чтоб сделать его отличным. Особенно обрати внимание на скрытые дубликаты. Удачи </font>

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [X]  выделены леммы в значениях столбца с целями получения кредита;
- [X]  описан процесс лемматизации;
- [X]  данные категоризированы;
- [X]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.